<a href="https://colab.research.google.com/github/JooyoungO/machineLearning/blob/main/05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

일반적인 모델 생성 방법

1. 훈련 세트로 모델 생성
  * 하이퍼 파라미터 조절

2. 테스트 세트로 모델을 평가

  1. 성능이 만족스러우면 모델 생성 완료
  2. 성능이 만족스럽지 않으면 위 과정을 반복

이러한 과정을 반복하다보면 결국 모델이 훈련 세트 뿐만 아니라 테스트 세트에도 맞추어짐

**검증 세트**

모델을 만들고 나서 테스트 세트는 딱 한번 사용되는 것이 좋음

따라서 추가로 검증세트를 나눔

대략 훈련 세트 60% 검증 세트 20% 테스트 세트 20%

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트와 검증 세트의 평가 점스를 이용하여 최적화

최적화 완료 후 테스트 세트로 모델을 최종 평가

In [7]:
print(dt.score(test_input, test_target))

0.8569230769230769


**검증 세트의 문제점**

검증 세트로 인해 훈련 세트가 줄어듬

보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐

검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불완정함

이러한 문제를 보완해야 함

**교차 검증**

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00861812, 0.0072813 , 0.00750566, 0.00747418, 0.00710869]), 'score_time': array([0.00107551, 0.00106835, 0.00107956, 0.00124192, 0.0010426 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


교차 검증의 최종점수

이름은 test_score 이지만 테스트 점수가 아닌 검증 정수임

In [9]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의 할 점

cross_validate() 는 훈련 세트를 섞어 폴드를 나누지 않음

만약 train_test_split()를 하지 않았다면 분할기를 지정해야함

cross_validate() 는 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 StratifiedKFold를 사용

섞지 않는 것이 디폴트 이므로 따로 설정해줘야 함

In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


10 폴드 교차 검증을 수행

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01064968, 0.00826669, 0.00846171, 0.01043415, 0.00803804,
       0.00815821, 0.01200676, 0.00919151, 0.00818706, 0.00846672]), 'score_time': array([0.00110221, 0.00096107, 0.00098443, 0.00131965, 0.00099921,
       0.00095677, 0.00175476, 0.00103855, 0.00102186, 0.00104904]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
0.8574181117533719


cross_validate() 는 자체적으로 모델을 훈련시키기는 하나 모델 파라미터나 하이퍼 파라미터를 찾는 것이 아니라 모델의 점수 검증을 위해서 사용함


**하이퍼파라미터 튜닝**


하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

ex. 결정 트리 모델에서는 max_depth, min_samples_split, min_impurity_decrease

반대로 모델이 학습하는 파라미터는 모델 파라미터

예를 들어, 결정 트리 모델에서는 분할 변수와 분할점

튜닝 작업은 기본값을 그대로 사용해서 모델을 훈련한 후, 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔가면서 최적의 값을 찾음

문제는 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안됨

모든 매개변수를 동시에 바꿔가면서 최적의 값을 찾아야 함

매개변수가 많아질수록 최적의 값들을 찾는 것이 복잡함

min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터 수

ex. 한번 나눴는데 한 집단에 2개의 데이터가 들어있다면 더 이상 나누지 않는 것

만약에 1이라고 끝까지 계속 나누기 때문에 과대적합의 가능성이 있음

**그리드 서치**

하이퍼파라미터 탐색과 교차검증을 한번에 수행

cross_validate() 함수를 호출할 필요가 없음

In [12]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


**그리드 서치를 이용한 최적화 과정 정리**

1. 탐색할 매개변수를 지정

2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음

  * 이 조합은 그리드 서치 객체에 저장

3. 그리드 서치는 최상으 매개변수에서 (교차검증에 사용한 훈련 세트가 아닌) 전체 훈련세트를 사용해 최종 모델을 훈련

  * 이 모델은 그리드 서치 객체에 저장

In [19]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


각 매개변수에서 수행한 교차 검증의 평균 점수의 최대 값

In [22]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


찾은 파라미터로 모델을 생성하고 테스트 점수 확인

In [23]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


**랜덤 서치**

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음

너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음

이럴 때 랜덤 서치를 사용하면 좋음

매개변수 값의 목록이 아닌 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달함

**싸이 파이 scipy**

파이썬의 핵심 과학 라이브러리 중 하나

적분, 보간, 선형 댓, 확률 등을 포함한 수치 계산 전용 라이브러리

uniform과 randint 클래스

주어진 범윙서 고르게 값을 뽑음 -> 균등 분포에서 샘플링한다라고 함

randint 는 정수값을 uniform 은 실수값을 선택



In [24]:
from scipy.stats import uniform, randint

In [25]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 1, 3, 3, 4, 9, 8, 6, 5, 0])

In [26]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108,  98, 104, 105, 107, 102,  89,  96,  81, 110]))

In [27]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.98947197, 0.77338   , 0.76546028, 0.79797909, 0.82661648,
       0.47532744, 0.86032342, 0.08962511, 0.16856415, 0.36299288])

In [28]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [29]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x781b4602ba60>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x781b460719c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x781b4602be50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x781b46073400>},
                   random_state=42)

In [30]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [32]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86
